In [13]:
import os
from dotenv import load_dotenv

from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "user", "content": "Hello!"},
    ],
    temperature=0.0,
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [15]:
import json

with open("./resource/reviews.json", "r") as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '9살 11살아이와함께 해돋이보러 첫방문한 그랑베이호텔 너무좋았어요.\n아이와 함께 여행을 다니다보니 청결이가장 중요한데  침구도 깨끗하고, 구석구석 먼지없이 깨끗했고요 욕실도 깔끔했습니다. 아이들이 있어 환기가 필수인데룸마다 발코니도 있어서 환기하기도 좋았어요. \n침구류도 쾌적해서 꿀잠잤네요.\n아이들도 엄마 여기좋다 또오고싶다 이러면서 만족스러워하니 엄마로써 아이들웃음에 더 좋았네요. \n바로앞이 바다여서 바다보며 힐링도하고요 시원하게 부셔지는 파도가 참 인상적이였어요.  \n해돋이 명소더라고요. 해변모래도 고와서 아이들이 뛰어다니며 놀기좋았어요. \n다음에는 인피니트 풀도 이용해보러 또 재방문 하려고요 ^^',
  'stars': 5,
  'date': '2025.01.01'},
 {'review': '신상호텔이라 그런지 깨끗하고 좋아요! 수영장도 잘 이용했습니다~ 카드키가 없고 도어락 방식이라 넘 편하구 좋았어요~^^*',
  'stars': 5,
  'date': '2024.08.25'},
 {'review': '아직 신축이라 싸게 잘 놀고 온것 같아요\n아이들과 즐거운 시간이였습니다~^^',
  'stars': 5,
  'date': '2024.08.10'}]

### 좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [ ]:
good_cnt = 0
bad_cnt = 0

for review in review_list:
    if review["stars"] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

print(f"좋은 평점: {good_cnt}, 나쁜 평점: {bad_cnt}")

좋은 평점: 435, 나쁜 평점: 200


### 리뷰 구분해서 저장

In [ ]:
good_reviews = []
bad_reviews = []

for review in review_list:
    if review["stars"] == 5:
        good_reviews.append("[review_start]" + review["review"] + "[review_end]")
    else:
        bad_reviews.append("[review_start]" + review["review"] + "[review_end]")

print(good_reviews[:3])
print(bad_reviews[:3])

['[review_start]9살 11살아이와함께 해돋이보러 첫방문한 그랑베이호텔 너무좋았어요.\n아이와 함께 여행을 다니다보니 청결이가장 중요한데  침구도 깨끗하고, 구석구석 먼지없이 깨끗했고요 욕실도 깔끔했습니다. 아이들이 있어 환기가 필수인데룸마다 발코니도 있어서 환기하기도 좋았어요. \n침구류도 쾌적해서 꿀잠잤네요.\n아이들도 엄마 여기좋다 또오고싶다 이러면서 만족스러워하니 엄마로써 아이들웃음에 더 좋았네요. \n바로앞이 바다여서 바다보며 힐링도하고요 시원하게 부셔지는 파도가 참 인상적이였어요.  \n해돋이 명소더라고요. 해변모래도 고와서 아이들이 뛰어다니며 놀기좋았어요. \n다음에는 인피니트 풀도 이용해보러 또 재방문 하려고요 ^^[review_end]', '[review_start]신상호텔이라 그런지 깨끗하고 좋아요! 수영장도 잘 이용했습니다~ 카드키가 없고 도어락 방식이라 넘 편하구 좋았어요~^^*[review_end]', '[review_start]아직 신축이라 싸게 잘 놀고 온것 같아요\n아이들과 즐거운 시간이였습니다~^^[review_end]']
['[review_start]편히 잘 쉬었습니다!![review_end]', '[review_start]청결한 객실이 마음에 들고 수영장이 좋아요[review_end]', '[review_start]최악입니다....\n조식도 최악 복도 방음도 최악....\n그냥 돈없어서 여기 방문한 제가 바보죠 뭐[review_end]']


In [ ]:
good_reviews_text = "\n".join(good_reviews)
bad_reviews_text = "\n".join(bad_reviews)

print(good_reviews_text[:100])
print(bad_reviews_text[:100])

[review_start]9살 11살아이와함께 해돋이보러 첫방문한 그랑베이호텔 너무좋았어요.
아이와 함께 여행을 다니다보니 청결이가장 중요한데  침구도 깨끗하고, 구석구석 먼지없이
[review_start]편히 잘 쉬었습니다!![review_end]
[review_start]청결한 객실이 마음에 들고 수영장이 좋아요[review_end]
[review_sta


### 전처리 함수 작성

In [ ]:
import datetime
from dateutil import parser


def preprocess_reviews(path="./resource/reviews.json"):
    with open(path, "r", encoding="utf-8") as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=3 * 30)

    for review in review_list:
        review_date_str = review["date"]
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        # 3개월 이전 리뷰는 제외
        if review_date < date_boundary:
            continue

        # 평점 구분해서 리뷰 저장
        if review["stars"] == 5:
            reviews_good.append("[REVIEW_START]" + review["review"] + "[REVIEW_END]")
        else:
            reviews_bad.append("[REVIEW_START]" + review["review"] + "[REVIEW_END]")

    reviews_good_text = "\n".join(reviews_good)
    reviews_bad_text = "\n".join(reviews_bad)

    return reviews_good_text, reviews_bad_text


good, bad = preprocess_reviews()

print(good[:100])
print(bad[:100])

[REVIEW_START]파노라마룸 최고에요^^!
수영장도 예쁨~~!![REVIEW_END]
[REVIEW_START]깨끗하고 좋았습니다
안좋은평보고 걱정했는데 저는 거슬리는게 하
[REVIEW_START]편히 잘 쉬었습니다!![REVIEW_END]
[REVIEW_START]청결한 객실이 마음에 들고 수영장이 좋아요[REVIEW_END]
[REVIEW_STA


### 평가용 함수 작성

#### 평가 기준 설정

- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

#### 평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [ ]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[{"role": "user", "content": eval_prompt}],
        temperature=0.0,
    )

    return completion

### 베이스라인 모델

In [57]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [ ]:
reviews, _ = preprocess_reviews(path="./resource/reviews.json")


def summarize(reviews, prompt, temperature=0.0, model="gpt-3.5-turbo-0125"):
    prompt = prompt + "\n\n" + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )

    return completion


print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

67
1. 깨끗하고 좋았고, 거슬리는 것이 없었으며, 다시 갈 의향이 있음.
2. 수영장이 아쉬웠지만, 직원 친절하고 주변이 공사중이어도 만족함.
3. 16층 바다 뷰가 좋고, 가족 여행으로 좋았음.
4. 양양여행으로 편안하게 쉬다 왔으며, 부대시설 다양하고 해수욕장이 가까워 좋았음.
5. 낙산해변이 핫한 곳이라고 하며, 주변 맛집과 생맥주가 좋았음.


In [ ]:
summary_real_20251002 = "해변과 가까운 위치에 있는 숙소로, 깨끗한 객실과 다양한 부대시설이 마련되어 있습니다. 루프탑 수영장과 인피니티 풀이 있어 가족 단위 여행객에게 적합하며, 주변에 편의시설과 맛집도 많아 편리합니다. 그러나 엘리베이터 수가 부족하여 이용 시 불편함이 있을 수 있으며, 성수기에는 주차 공간도 협소할 수 있습니다. 전반적으로 청결 상태가 우수하고, 재방문 의사를 가진 이용자들이 많습니다."

In [ ]:
print(
    pairwise_eval(
        reviews,
        summarize(reviews, PROMPT_BASELINE).choices[0].message.content,
        summary_real_20251002,
    )
    .choices[0]
    .message.content
)

In evaluating the summaries provided by the two assistants, it is clear that both responses capture key aspects of the user reviews, but they differ in depth and detail.

Assistant A presents a list format that summarizes individual reviews, highlighting specific points such as cleanliness, views, and staff friendliness. However, it lacks a cohesive narrative and does not synthesize the information into a comprehensive overview. While it does mention various positive aspects, it does not address any potential drawbacks or provide a balanced view.

On the other hand, Assistant B offers a more integrated summary that encapsulates the overall experience of the accommodations. It mentions the cleanliness, proximity to the beach, and the availability of various facilities, while also acknowledging the issues with elevator availability and parking during peak season. This response provides a more rounded perspective, making it more helpful for someone looking to understand both the strengths

### 대규모 평가 스크립트

- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [ ]:
eval_count = 10

summaries_baseline = [
    summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content
    for _ in range(eval_count)
]
summaries_baseline

['1. 깨끗하고 좋아서 다시 갈 예정입니다.\n2. 수영장이 조금 아쉽지만, 직원분들 친절하고 주변 환경도 괜찮았습니다.\n3. 16층 바다뷰 객실이 좋았고 가족 여행으로 즐거운 시간 보냈습니다.\n4. 양양여행을 편하게 즐길 수 있었고 전체적으로 만족스러웠습니다.\n5. 가격대비 좋은 숙소이지만 엘리베이터 대기시간이 길어 조금 불편함을 느꼈습니다.',
 '1. 깨끗하고 편안한 숙소로 좋은평을 받았으며, 다양한 부대시설과 바다 뷰가 매력적이었다.\n2. 수영장이 아쉽긴했지만, 친절한 직원과 바다 근처 위치, 편의시설이 좋았다는 후기.\n3. 가족여행으로 이용한 숙소는 16층 뷰와 해변 근접이 좋았다.\n4. 2박3일 양양여행 후 맘에 드는 뷰와 수영장, 부대시설에 대한 긍정적인 리뷰.\n5. 뷰가 좋고 방 상태도 좋았으며, 조식과 직원의 서비스에 만족했다.',
 '1. 깨끗하고 좋았으며, 다음에도 방문할 의향이 있습니다.\n2. 수영장은 아쉬움이 있지만, 직원 친절하고 주변에 편의시설이 많아서 아이와 즐거운 시간을 보낼 수 있었습니다.\n3. 16층 뷰가 좋고, 바다로 바로 접근할 수 있어서 가족 여행으로 즐거운 시간을 보냈습니다.\n4. 부대시설과 해수욕장이 가까워서 편하게 여행할 수 있었으며, 엘리베이터 대기시간이 조금 길었던 것을 제외하고 만족했습니다.\n5. 수영장은 좋았지만, 규칙과 엘리베이터가 불편했으며 전반적으로는 만족스러운 휴가를 보냈습니다.',
 '청결하고 좋았으며 거슬리는 점이 없었고, 다시 방문하고 싶어했으며 수영장이 아쉬웠지만 아이들과 즐거운 시간을 보내고 좋은 서비스를 받았다. \n\n낙산해변 근처이고 뷰와 깨끗한 객실로 가족 여행을 만족시켰다. \n\n양양여행에서 편안하게 쉴 수 있었고, 객실 뷰와 부대시설을 칭찬하였으나 방음과 직원의 친절에 살짝 아쉬움이 있었다. \n\n낙산해변 근처로 환경이 좋아 많은 매력을 느꼈고, 다양한 부대시설로 재방문을 고려했으나 수영장에서의 엘리베이터 대기시간이 조금 아쉽다고 느꼈다. \n\n루프탑 수영

In [ ]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if "[[A]]" in verdict_text:
            a_cnt += 1
        elif "[[B]]" in verdict_text:
            b_cnt += 1
        elif "[[C]]" in verdict_text:
            draw_cnt += 1
        else:
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt


wins, losses, ties = pairwise_eval_batch(
    reviews,
    summaries_baseline,
    [summary_real_20251002 for _ in range(len(summaries_baseline))],
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:50<00:00,  5.08s/it]

Wins: 3, Losses: 7, Ties: 0


### 모델 고도화1
- 조건 명시

In [ ]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [
    summarize(reviews, prompt, temperature=1.0).choices[0].message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real_20251002 for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:45<00:00,  4.52s/it]

Wins: 4, Losses: 6, Ties: 0


### 모델 고도화 2 
- 입력 데이터의 품질 증가

In [ ]:
import datetime
from dateutil import parser


def preprocess_reviews(path="./resource/reviews.json"):
    with open(path, "r", encoding="utf-8") as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=3 * 30)

    filtered_cnt = 0
    for review in review_list:
        review_date_str = review["date"]
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(review["review"]) < 30:
            filtered_cnt += 1
            continue

        if review["stars"] == 5:
            reviews_good.append("[REVIEW_START]" + review["review"] + "[REVIEW_END]")
        else:
            reviews_bad.append("[REVIEW_START]" + review["review"] + "[REVIEW_END]")

    reviews_good = reviews_good[: min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[: min(len(reviews_bad), 50)]

    reviews_good_text = "\n".join(reviews_good)
    reviews_bad_text = "\n".join(reviews_bad)

    return reviews_good_text, reviews_bad_text


reviews, _ = preprocess_reviews()

In [ ]:
eval_count = 10
summaries = [
    summarize(reviews, prompt, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real_20251002 for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:50<00:00,  5.01s/it]

Wins: 5, Losses: 5, Ties: 0


### 모델 고도화 3 
- Few-Shot Prompting

In [ ]:
reviews_1shot, _ = preprocess_reviews("./resource/ninetree_pangyo.json")
summary_1shot = (
    summarize(reviews_1shot, prompt, temperature=0.0, model="gpt-4-turbo-2024-04-09")
    .choices[0]
    .message.content
)
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real_20251002 for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:46<00:00,  4.61s/it]

Wins: 3, Losses: 7, Ties: 0


In [67]:
summaries

['전반적으로 숙소는 깨끗하고 좋았습니다. 걱정했던 점은 없었고, 또 방문하고 싶은 숙소였습니다. 주변에 공사중이라는 점은 상관없었어요~ 숙소를 즐길 수 있는 편안한 시설들도 다양하게 제공되고 있어서 만족스러웠습니다~요. 재밌게 보내고 싶은 가족 여행이었습니다~ 호텔 직원분들도 친절하셨고, 수영장이 아이들과 재미있게 놀기 좋았어요. 이용 객 수가 많은 것에 비해 엘리베이터 대기 시간이 오래 걸렸지만, 다른 시설들은 즐길만했습니다. 종합적으로 만족스러운 숙박이었습니다~ ',
 '1. 깨끗하고 좋은 숙소입니다. 걱정할 필요 없이 즐기실 수 있을 것 같아요.\n2. 수영장이 한 번만 이용할 수 있는 점이 아쉽지만, 아이와 함께 재미난 시간을 보낼 수 있었어요. 직원분들이 친절하셔서 기분 좋았습니다.\n3. 가족 여행으로 머물었는데, 16층에 위치하여 뷰도 좋고 해변 접근성이 좋았습니다. 쾌적한 휴식을 취할 수 있었어요. \n4. 전반적으로 편안한 양양여행을 즐겼다는 리뷰들이 많네요. 향후에도 재방문할 의사가 있는 것으로 보입니다.\n5. 최상의 직원 친절함과 객실 컨디션을 자랑하는 숙소입니다. 다양한 부대시설도 만족스러웠지만 엘리베이터 대기 시간이 조금 길었습니다.',
 '전반적으로 좋았고, 숙소는 깨끗했습니다. 걱정했던 것이 하나도 없었어요. 또 다시 방문할 의향이 있습니다. 루프탑 수영장은 아이들에게 좋았지만, 엘리베이터 대기시간이 조금 길었어요. 수영장이 생각보다 작았지만, 전망이 너무 좋았습니다. 해당 숙소는 가족 휴가에 적합하며, 전반적으로 만족스러운 시설이었습니다.',
 '다양한 리뷰를 통해 숙소에 대한 평이 상당히 긍정적임을 알 수 있습니다. 숙소는 청결하고 위치가 좋아 해수욕을 즐길 수 있으며, 부대시설도 매우 만족스러운 것으로 보입니다. 다만 엘리베이터 대기시간이 긴 등 일부 불편함이 있으나 전반적으로는 만족도가 높았던 것으로 보입니다. 호텔 주변에는 맛집이나 놀거리가 다양하게 있어 즐거운 휴가를 보낼 수 있었다는 평이 많습니다. 호텔의 깨끗한 객실

In [ ]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt_1shot, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real_20251002 for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [01:14<00:00,  7.49s/it]

Wins: 2, Losses: 8, Ties: 0


In [69]:
summaries

['이 숙소는 바다뷰와 깨끗한 객실로 많은 방문객들에게 좋은 평가를 받고 있습니다. 수영장 부대시설이 좋고, 주변 편의시설도 다양하게 위치해 있어 휴가를 즐기기에 좋은 장소로 평가받고 있습니다. 그러나 일부 방문객들은 엘리베이터가 협소하고 불편하다는 의견을 제기했습니다. 전반적으로 만족도가 높은 숙소로, 다양한 부대시설과 깨끗한 객실 상태로 재방문 의사도 높은 것으로 보입니다.',
 '이 숙소는 바다 뷰와 전반적으로 깨끗한 객실 컨디션으로 많은 방문객들에게 좋은 평가를 받고 있습니다. 부대시설도 다양하게 구비되어 있어 가족 여행객들이 편리하게 놀거리를 찾을 수 있습니다. 하지만 엘리베이터가 협소하여 이용에 어려움이 있고, 화장실의 문제 등 몇 가지 부분에서 향상이 필요한 부분이 있습니다. 전반적으로는 가격대비 좋은 숙소로 추천됩니다.',
 '숙소는 전체적으로 청결하고 편안한 환경을 제공하며, 특히 수영장 시설이 인기가 많다는 평가를 받고 있습니다. 주변에 다양한 식당과 편의시설이 위치해 있어 편리함을 더하고, 방문객들은 친절한 서비스와 쾌적한 객실 상태에 만족을 표현하고 있습니다. 다만, 엘리베이터 혼잡이나 일부 객실의 화장실 문제 같은 일부 부분에 대한 불편함이 언급되고 있습니다. 전반적으로 이 숙소는 가족, 친구 등과 함께 즐길 수 있는 부대시설을 갖추고 있어 권장됩니다.',
 '이 숙소는 위치가 바다와 가까워서 해수욕하기에 좋다는 평가를 받고 있으며, 객실 컨디션과 시설 등도 만족스러운 편입니다. 전반적으로 깔끔하고 친절한 서비스로 고객들에게 만족을 준다는 의견이 있습니다. 다만, 엘리베이터가 혼잡하고 긴 대기시간 때문에 불편함을 느끼는 손님들도 있었습니다. 또한, 몇몇 객실에서는 변기와 샤워공간이 협소하다는 언급도 있습니다. 종합적으로는 가족이나 친구들과 함께하는 여름 휴가로 적합한 숙소로 추천되고 있습니다.',
 '이 숙소는 깨끗하고 아늑한 객실을 제공하며, 수영장이 인기가 많아 가족 단위로 방문하는 곳으로 좋은 평가를 받았습니다. 주변에 편의시설이 

In [ ]:
reviews_1shot, _ = preprocess_reviews("./resource//ninetree_pangyo.json")
summary_1shot = (
    summarize(reviews_1shot, prompt, temperature=0.0, model="gpt-4-turbo-2024-04-09")
    .choices[0]
    .message.content
)

In [ ]:
reviews_2shot, _ = preprocess_reviews("./resource/ninetree_yongsan.json")
summary_2shot = (
    summarize(
        reviews_2shot, prompt_1shot, temperature=0.0, model="gpt-4-turbo-2024-04-09"
    )
    .choices[0]
    .message.content
)

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [
    summarize(reviews, prompt_2shot, temperature=1.0, model="gpt-3.5-turbo-0125")
    .choices[0]
    .message.content
    for _ in range(eval_count)
]
wins, losses, ties = pairwise_eval_batch(
    reviews, summaries, [summary_real_20251002 for _ in range(len(summaries))]
)
print(f"Wins: {wins}, Losses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:53<00:00,  5.36s/it]

Wins: 6, Losses: 4, Ties: 0


In [72]:
summaries

['이 숙소는 바다 뷰를 감상할 수 있고, 객실 컨디션이 깨끗하며 부대시설이 만족스러웠습니다. 하지만 엘리베이터 부족으로 불편함을 느낀 점이 있었고, 화장실 세면대의 물이 잘 내려가지 않는 등 몇 가지 문제점이 있었던 것으로 보입니다. 다양한 편의시설과 놀거리가 있어 가족 휴가로 이용하기에 적합한 숙소라는 의견이 많습니다. 요약 결과는 다소 변화가 있는 것으로 판단됩니다. 별도의 평가 지표를 설정하여 테스트해보세요.',
 '요약 결과:\n다양한 여행객들의 리뷰를 종합해본 결과, 대부분의 방문객들이 깨끗하고 쾌적한 숙박환경과 수영장 시설을 칭찬하고 있습니다. 또한, 주변 식당과 편의시설의 편리함도 언급되었습니다. 그러나 일부 리뷰에서는 엘리베이터의 혼잡과 화장실 문제, 그리고 어메니티 부족을 언급하는 경우도 있었습니다. 전반적으로는 숙박 환경과 서비스에 만족한 고객이 많았으며, 높은 재방문 의사를 밝히는 의견이 많았습니다. 위치와 숙소 청결을 중시하는 분들에게 적합한 숙박시설로 평가됩니다.',
 '다수의 리뷰들에서는 숙소가 깨끗하고 쾌적하며 위치가 좋다는 긍정적인 평가를 받았습니다. 방음이나 엘리베이터 문제, 수영장 이용 규칙 등 일부 불편한 점들도 언급되었지만, 전반적으로는 숙소의 청결도와 부대시설에 대한 만족도가 높은 것으로 보입니다. 숙소 위치와 해변 근접성, 객실 내용, 서비스 수준 등을 고려하면 다양한 연령대와 가족 단위에게 적합한 숙소로 평가됩니다. 요약하면, 깨끗하고 쾌적한 환경의 숙소로 위치와 부대시설이 만족스러운 것으로 전반적인 리뷰 결론을 이끌어 낼 수 있을 것입니다.',
 '다양한 리뷰를 요약해 보겠습니다. \n\n1. 숙소는 청결하고 수영장이 좋아서 인기가 많습니다. 직원들도 친절하고 주변에 다양한 식당과 편의시설이 위치해 있어 편리함을 더해줍니다. 재방문 의사가 높은 편입니다.\n2. 객실은 깨끗하고 쾌적하며, 수영장이나 인근 편의시설을 이용해 즐거운 휴가를 보낼 수 있습니다. 일부 엘레베이터가 혼잡하여 불편함을 느낀 리뷰도 있습니다.\n

# 결과가 항상 좋아지지는 않는다